# Autoencoder

In [2]:
import numpy as np
import urllib.request
from gensim.utils import simple_preprocess

# Download GloVe embeddings
# glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
# urllib.request.urlretrieve(glove_url, "glove.6B.zip")

# import zipfile
# with zipfile.ZipFile("glove.6B.zip", "r") as zip_ref:
#     zip_ref.extractall("glove.6B")

In [ ]:
# Example custom dataset
custom_sentences = [
    'african_buffalo', 'alligator', 'amphibian', 'amur_leopard', 
    'ants', 'bear', 'bird', 'blue_whale', 'bobcat', 'cat', 'chimp', 
    'chimpanzee', 'cow', 'dog', 'dolphin', 'domestic_water_buffalo', 
    'eagle', 'elephant', 'fish', 'frog', 'giant', 'giant_panda', 'goat', 
    'gorilla', 'hen', 'horse', 'killer_whale', 'lion', 'lizard', 'monkey', 
    'mouse', 'orangutan', 'ostrich', 'ox', 'panda', 'polar_bear', 'rabbit', 
    'rat', 'rhino', 'rhinoceros', 'rhinoceroses', 'seal', 'sealskin', 
    'siamese_cat', 'skunk', 'spider_monkey', 'squirrel', 'tiger', 'turtle', 
    'walrus', 'whale', 'bird', 'fish', 'lion', 'tiger', 'bull'
]

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

embeddings_index = load_glove_embeddings("glove.6B/glove.6B.50d.txt")  # Using 50d GloVe embeddings
print(f"embeddings_index : {type(embeddings_index)} \n")

# Convert sentences to embeddings
def sentence_to_embedding(sentence, embeddings_index):
    words = simple_preprocess(sentence)
    valid_words = [embeddings_index[word] for word in words if word in embeddings_index]
    if valid_words:
        return np.mean(valid_words, axis=0)
    else:
        return np.zeros(50)

sentence_embeddings = np.array([sentence_to_embedding(sentence, embeddings_index) for sentence in custom_sentences])
print(f"sentence_embeddings : {type(embeddings_index)} \n")

In [5]:
l1 = set([
    'african_buffalo', 'alligator', 'amphibian', 'amur_leopard', 
    'ants', 'bear', 'bird', 'blue_whale', 'bobcat', 'cat', 'chimp', 
    'chimpanzee', 'cow', 'dog', 'dolphin', 'domestic_water_buffalo', 
    'eagle', 'elephant', 'fish', 'frog', 'giant', 'giant_panda', 'goat', 
    'gorilla', 'hen', 'horse', 'killer_whale', 'lion', 'lizard', 'monkey', 
    'mouse', 'orangutan', 'ostrich', 'ox', 'panda', 'polar_bear', 'rabbit', 
    'rat', 'rhino', 'rhinoceros', 'rhinoceroses', 'seal', 'sealskin', 
    'siamese_cat', 'skunk', 'spider_monkey', 'squirrel', 'tiger', 'turtle', 
    'walrus', 'whale', 'bird', 'fish', 'lion', 'tiger', 'bull'
])

l1 = list(l1)

sorted(l1)

['african_buffalo',
 'alligator',
 'amphibian',
 'amur_leopard',
 'ants',
 'bear',
 'bird',
 'blue_whale',
 'bobcat',
 'bull',
 'cat',
 'chimp',
 'chimpanzee',
 'cow',
 'dog',
 'dolphin',
 'domestic_water_buffalo',
 'eagle',
 'elephant',
 'fish',
 'frog',
 'giant',
 'giant_panda',
 'goat',
 'gorilla',
 'hen',
 'horse',
 'killer_whale',
 'lion',
 'lizard',
 'monkey',
 'mouse',
 'orangutan',
 'ostrich',
 'ox',
 'panda',
 'polar_bear',
 'rabbit',
 'rat',
 'rhino',
 'rhinoceros',
 'rhinoceroses',
 'seal',
 'sealskin',
 'siamese_cat',
 'skunk',
 'spider_monkey',
 'squirrel',
 'tiger',
 'turtle',
 'walrus',
 'whale']